In [1]:
%load_ext autoreload
%autoreload 2

##### Please run this code with the kernel reinvent.v3.2

In [2]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

### First, we need to install the custom reinvent scoring package to support the Bradley-Terry model

In [3]:
# print python version
import sys

# Print Python version
print(f"Python version: {sys.version}")

Python version: 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]


In [4]:
! pip show reinvent_scoring

Name: reinvent-scoring
Version: 0.0.73
Summary: Scoring functions for Reinvent
Home-page: https://github.com/MolecularAI/reinvent-scoring.git
Author: MolecularAI
Author-email: patronov@gmail.com
License: UNKNOWN
Location: /home/springnuance/reinvent-hitl/reinvent-scoring
Requires: 
Required-by: 


In [5]:
# ! conda install scikit-learn=0.21.3

##### If there already exists reinvent_scoring, we should uninstall it

In [6]:
! pip uninstall -y reinvent_scoring

Found existing installation: reinvent-scoring 0.0.73
Uninstalling reinvent-scoring-0.0.73:
  Successfully uninstalled reinvent-scoring-0.0.73


##### Now we install the custom reinvent scoring package
##### The flag -e means that the package is installed in editable mode, so that changes to the code will be immediately available without reinstalling the package. All package info is stored in the setup.py file.

In [7]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-chemistry"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-models"

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reinvent-scoring
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-chemistry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-chemistry
    Found existing installation: reinvent-chemistry 0.0.51
    Uninstalling reinvent-chemistry-0.0.51:
      Successfully uninstalled reinvent-chemistry-0.0.51
  Running setup.py develop for reinvent-chemistry
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-models
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-models
    Found existing installation: reinvent-models 0.0.15rc1
    Uninstalling reinvent-models-0.0.15rc1:
      Successfully uninstalled reinvent-models-0.0.15rc1
  Running setup.py develop for reinvent-models


In [8]:
# ! conda install -y scikit-learn=0.21.3
! pip list | grep reinvent_scoring

In [14]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.acquisition import predict_proba_from_model

def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
def run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each iteration
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs # number of epochs for training the model in each HITL iteration
        ):

    np.random.seed(seed)
    rng = default_rng(seed)
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = output_dir
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    for REINVENT_round in range(1, num_rounds + 1):
        check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")
        for HITL_iteration in range(1, num_iters + 1):
            check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}")
    
    # Copy the pretrained model to the first REINVENT round/ HITL_iteration_1
    shutil.copy2(f"{model_pretrained_path}", f"{output_dir}/REINVENT_round_1/HITL_iteration_1")

    # multi-parameter optimization (MPO) loop
    print(f"\nRunning DRD2 (one objective) with rounds {num_rounds}, iters {num_iters}, queries {num_queries}, seed {seed}")
    print(f"Results will be saved at {output_dir}")
    
    base_training_dataset_outputs = load_drd2_dataset(feedback_type=feedback_type, 
                                                 data_path=base_training_dataset_path)

    
    print("Loading initial training and testing datasets successfully")

    # ########################### REINVENT rounds ######################################

    for REINVENT_round in range(1, num_rounds + 1):

        print("=====================================")
        print(f"REINVENT round = {REINVENT_round}")

        REINVENT_round_output_dir = f"{output_dir}/REINVENT_round_{REINVENT_round}"
        
        configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                                                        REINVENT_round_output_dir, conf_filename)

        print(f"Creating config file: {configuration_JSON_path}.")

        configuration = json.load(open(f"{configuration_JSON_path}"))

        current_model_path = f"{REINVENT_round_output_dir}/HITL_iteration_1/{model_pretrained_name}"

        configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

        # write the updated configuration file 

        with open(configuration_JSON_path, 'w') as f:
            json.dump(configuration, f, indent=4, sort_keys=True)
    
        print("Run REINVENT")                
        command = f"{reinvent_env}/bin/python"
        script = f"{reinvent_dir}/input.py"
        stderr_file = f"{REINVENT_round_output_dir}/run.err"
        stdout_file = f"{REINVENT_round_output_dir}/run.out"

        # Construct the full command to run
        cmd = [command, script, configuration_JSON_path]
        # Open the file to which you want to redirect stderr and stdout
        with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
            # Execute the command
            result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
        # Check the result
        print("Exit code:", result.returncode)
        
        #############################################################################
        # REINVENT HAS OUTPUT THE RESULT in path f"{REINVENT_round_output_dir}/results" #
        #############################################################################
        
        # Get the high scoring molecules
        output_high_score = read_scaffold_result(f"{REINVENT_round_output_dir}/results/scaffold_memory.csv", 
                                                 choose_top_smiles=choose_top_smiles)
        
        scaffold_df = output_high_score["scaffold_df"]
        smiles = output_high_score["smiles"]
        
        # store molecule indexes selected for feedback
        selected_feedback = np.empty(0).astype(int)


        
        ########################### HITL_iteration in each REINVENT round #####################
        
        for HITL_iteration in range(1, num_iters + 1): # T number of HITL_iterations

            print("----------------------------------")
            print(f"HITL iteration = {HITL_iteration}")
            
            # Loading feedback model
            feedback_model_path = f"{REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/{model_pretrained_name}"
            feedback_model = load_feedback_model(feedback_type, feedback_model_path)
            
            print(f"The predicted scores are")
            predicted_scores = predict_proba_from_model(feedback_type, feedback_model, smiles)
            print(predicted_scores)

            ######################################################## 
            # Select queries number of smiles with Active Learning #
            ########################################################

            if len(smiles) > num_queries:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, num_queries, list(smiles), 
                                                  selected_feedback, acquisition, rng) 
            else:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, len(smiles), list(smiles), 
                                                  selected_feedback, acquisition, rng)
            
            print(f"Feedback idx at HITL iteration {HITL_iteration}: {new_queried_smiles_indices}")
            
            new_queried_smiles = [smiles[i] for i in new_queried_smiles_indices]
  
            selected_feedback = np.hstack((selected_feedback, new_queried_smiles_indices))

            new_queried_smiles_human_score = smiles_human_score(new_queried_smiles, sigma_noise)
            
            print(f"Human score at HITL iteration {HITL_iteration}: {new_queried_smiles_human_score}")
            
            # use the augmented training data to retrain the model
            new_queried_fps = np.array([compute_fingerprints(smiles) for smiles in new_queried_smiles])

            iteration_training_dataset_outputs = create_drd2_dataset(feedback_type, 
                                                                      new_queried_smiles, 
                                                                      new_queried_smiles_human_score,
                                                                      new_queried_fps)
            
            if feedback_type == "scoring":
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles'])}")
            else:
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles_1'])}")
            
            # combining the base training dataset with the new queried dataset
            base_training_dataset_outputs = combine_drd2_dataset(feedback_type, base_training_dataset_outputs, 
                                                                   iteration_training_dataset_outputs)
            
            if feedback_type == "scoring":
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles'])}")
            else:
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles_1'])}")
            
            # save augmented training data
            save_drd2_dataset(feedback_type, iteration_training_dataset_outputs, f"{REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/iteration_queried_data.csv")
            save_drd2_dataset(feedback_type, base_training_dataset_outputs, f"{REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            print(f"Saved augmented training data at {REINVENT_round_output_dir}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            # Retraining the feedback model using the augmented train set
            retrained_feedback_model = retrain_feedback_model(feedback_type, feedback_model, 
                                                              base_training_dataset_outputs,
                                                              training_epochs)
            
            if REINVENT_round != num_rounds:
                if HITL_iteration < num_iters:
                    # Moving on to the next iteration at the current round
                    feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration + 1}/{model_pretrained_name}"
                    torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)
                else:
                    # Moving to the first iteration at the next round
                    feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round + 1}/HITL_iteration_1/{model_pretrained_name}"
                    torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)
            else:
                if HITL_iteration < num_iters:
                    # Moving on to the next iteration at the current round
                    feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration + 1}/{model_pretrained_name}"
                    torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)
                else:
                    # Last iteration at the last round, we save to the final model
                    feedback_model_saving_path = f"{output_dir}/final_{model_pretrained_name}"
                    torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)

            print(f"Saved retrained feedback model at {feedback_model_saving_path}")

In [10]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


### Running score regression model

In [17]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.acquisition import predict_proba_from_model

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "scoring" # scoring, comparing, ranking

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
model_pretrained_name = "score_regression_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 1 # number of REINVENT optimization steps
batch_size = 32 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

# Please look at the thompson sampling code and fix it!
acquisition = "uncertainty" # acquisition: 'random', 'greedy', 'uncertainty', 'thompson', 

sigma_noise = 0.1 # noise level for simulated chemist's responses

num_queries = 5 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 20 # number of top molecules to choose from scaffold. 

training_epochs = 5 # number of epochs for training the model in each HITL iteration

output_dir = f"output_score_regression_R{num_rounds}_T{num_iters}_Q{num_queries}_{acquisition}_s{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

Running DRD2 (one objective) with rounds 2, iters 3, queries 5, seed 42
Results will be saved at output_score_regression_R2_T3_Q5_uncertainty_s0.1
Loading initial training and testing datasets successfully
REINVENT round = 1
Creating config file: output_score_regression_R2_T3_Q5_uncertainty_s0.1/REINVENT_round_1/config.json.
Run REINVENT


KeyboardInterrupt: 

### Running Bradley Terry model

In [20]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                    
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "comparing" # scoring, comparing, ranking

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
model_pretrained_name = "bradley_terry_model.pth"

num_rounds = 2 # number of rounds, corresponding to R in the paper
num_iters = 3 # number of iterations of showing molecules to the human for feedback at each round

REINVENT_n_steps = 5 # number of REINVENT optimization steps
batch_size = 32 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

# Please look at the thompson sampling code and fix it!
acquisition = "random" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.1 # noise level for simulated chemist's responses

num_queries = 10 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration
choose_top_smiles = 100 # number of top molecules to choose from scaffold. 

training_epochs = 5 # number of epochs for training the model in each HITL iteration

output_dir = f"output_bradley_terry_R{num_rounds}_T{num_iters}_Q{num_queries}_{acquisition}_s{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir,
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of molecules shown at each HITL_iteration to the human for feedback, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
)

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

Running DRD2 (one objective) with rounds 2, iters 3, queries 10, seed 42
Results will be saved at output_bradley_terry_R2_T3_Q10_random_s0.1
Loading initial training and testing datasets successfully
REINVENT round = 1
Creating config file: output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  122
----------------------------------
HITL iteration = 1
Loading Bradley Terry model successfully from output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_1/bradley_terry_model.pth
The predicted scores is


Found local copy...


[0.34 0.1  0.82 0.51 0.72 0.32 0.7  0.78 0.08 0.68 0.13 0.65 0.52 0.03
 0.74 0.49 0.06 0.16 0.18 0.01 0.44 0.33 0.83 0.41 0.14 0.64 0.84 0.39
 0.28 0.12 0.99 0.88 0.43 0.48 0.59 0.94 0.24 0.92 0.23 0.47 0.79 0.87
 0.07 0.73 0.15 0.76 0.8  0.95 0.09 0.75 0.05 0.26 0.58 0.97 0.31 0.29
 0.57 0.6  0.69 0.98 0.91 0.71 0.19 0.86 0.45 0.55 0.17 0.04 0.61 0.54
 0.81 0.5  0.67 0.02 0.63 0.46 0.11 0.85 0.21 0.93 0.4  0.38 0.22 0.77
 0.27 0.53 0.96 0.62 0.36 0.25 0.3  0.35 0.56 0.66 0.42 0.2  0.9  0.37
 0.89 0.  ]
Feedback idx at HITL iteration 1: [96 71  8 60 41 94 68  9 19 82]
Human score at HITL iteration 1: [0.053392034012538954, 0.0, 0.07228135485609186, 0.15839210369051424, 0.0, 0.0, 0.16000210790032912, 0.14176447952371987, 0.0, 0.05432364670038512]
New queried dataset size: 100
Combined dataset size: 124
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv
Epoch 1, Loss: 0.022981401284535727
Epoch 2, Loss

Found local copy...


[0.48 0.26 0.89 0.5  0.33 0.09 0.43 0.6  0.27 0.65 0.13 0.39 0.4  0.25
 0.49 0.45 0.08 0.58 0.22 0.12 0.15 0.42 0.51 0.69 0.46 0.95 0.72 0.07
 0.56 0.1  0.94 0.86 0.74 0.79 0.62 0.83 0.61 0.57 0.38 0.8  0.31 0.87
 0.01 0.23 0.02 0.98 0.88 0.92 0.17 0.41 0.11 0.54 0.35 0.84 0.18 0.29
 0.34 0.91 0.64 0.93 0.47 0.52 0.14 0.55 0.21 0.71 0.16 0.04 0.73 0.85
 0.77 0.75 0.24 0.05 0.67 0.76 0.3  0.7  0.03 0.99 0.36 0.68 0.44 0.9
 0.63 0.66 0.97 0.59 0.82 0.2  0.32 0.19 0.53 0.78 0.81 0.37 0.96 0.06
 0.28 0.  ]
Feedback idx at HITL iteration 2: [74 38 33 61 80 17 39 45 53 86]
Human score at HITL iteration 2: [0.0, 0.0, 0.024403938070629246, 0.0, 0.0, 0.0, 0.0, 0.036177740976959784, 0.0, 0.0]
New queried dataset size: 100
Combined dataset size: 224
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_2/iteration_combined_data.csv
Epoch 1, Loss: 0.08611303009092808
Epoch 2, Loss: 0.08617185242474079
Epoch 3, Loss: 0.08617958799004555
Epoch 4,

Found local copy...


[0.37 0.82 0.92 0.1  0.12 0.17 0.62 0.52 0.84 0.22 0.03 0.48 0.64 0.76
 0.56 0.29 0.31 0.75 0.02 0.78 0.25 0.46 0.65 0.55 0.73 0.98 0.51 0.32
 0.26 0.04 0.61 0.74 0.86 0.87 0.4  0.43 0.85 0.28 0.23 0.59 0.09 0.66
 0.   0.18 0.05 0.93 0.8  0.83 0.49 0.39 0.08 0.68 0.41 0.7  0.44 0.71
 0.19 0.96 0.2  0.9  0.45 0.15 0.6  0.13 0.24 0.63 0.01 0.34 0.89 0.79
 0.21 0.95 0.07 0.06 0.58 0.88 0.47 0.27 0.3  0.69 0.77 0.81 0.36 0.97
 0.53 0.38 0.72 0.35 0.91 0.14 0.16 0.42 0.5  0.94 0.57 0.67 0.99 0.11
 0.33 0.54]
Feedback idx at HITL iteration 3: [64 69 23 43 30 14 55 89 93  5]
Human score at HITL iteration 3: [0.16100094040468926, 0.0, 0.007227465099377878, 0.0, 0.0, 0.013044193374368563, 0.0, 0.037629686311089906, 0.0, 0.04351853236876942]
New queried dataset size: 100
Combined dataset size: 324
Saved augmented training data at output_bradley_terry_R2_T3_Q10_random_s0.1/REINVENT_round_1/HITL_iteration_3/iteration_combined_data.csv
Epoch 1, Loss: 1.0355503112077713
Epoch 2, Loss: 1.035826787352

KeyboardInterrupt: 

### Running Rank ListNet model

In [22]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
restart = False # If restart is True, we would rerun everything
                # If restart is False, we would continue from the latest found HITL_iteration
                
# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "ranking" # scoring, comparing, ranking

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"
base_testing_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_testing_data.csv"
model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
model_pretrained_name = "rank_listnet_model.pth"

num_rounds = 10 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 25 # number of REINVENT optimization steps

# WARNING: CHOOSING LARGER BATCH SIZE WOULD EXPONENTIALLY INCREASE THE NUMBER OF COMBINATIONS
# BETTER KEEP IT AT 32 OR 64
# For example, REINVENT would fail and return exit code -9 if it is 80

batch_size = 32 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

# Please look at the thompson sampling code and fix it!
acquisition = "random" # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 

sigma_noise = 0.0 # noise level for simulated chemist's responses

num_queries = 20 # number of molecules, pairs or a set of molecules, depending on the task, 
                 # shown to the simulated chemist at each HITL_iteration
choose_top_smiles = 64 # number of top molecules to choose from scaffold. 

training_epochs = 1 # number of epochs for training the model in each HITL iteration

output_dir = f"output_rank_listnet_R{num_rounds}_T{num_iters}_Q{num_queries}_{acquisition}_s{sigma_noise}"

run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        base_testing_dataset_path, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1/HITL_iteration_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
        REINVENT_n_steps, # number of REINVENT optimization steps
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top molecules to choose from scaffold
        training_epochs, # number of epochs for training the model in each HITL iteration
)


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

Running DRD2 (one objective) with rounds 10, iters 5, queries 20, seed 42
Results will be saved at output_rank_listnet_R10_T5_Q20_random_s0.0
Loading initial training and testing datasets successfully
REINVENT round = 1
Creating config file: output_rank_listnet_R10_T5_Q20_random_s0.0/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  789
----------------------------------
HITL iteration = 1
Loading Rank ListNet model successfully from output_rank_listnet_R10_T5_Q20_random_s0.0/REINVENT_round_1/HITL_iteration_1/rank_listnet_model.pth
The predicted scores is


Found local copy...


[0.82420208 0.39460659 0.36695682 0.78187404 0.73937532 0.81430278
 0.7859703  0.58371736 0.68032087 0.72708653 0.71650452 0.61665813
 0.78033794 0.60488138 0.51903055 0.72742789 0.68322239 0.78306878
 0.58201058 0.62160778 0.65864482 0.78153268 0.53985322 0.71309097
 0.77419355 0.76531831 0.68373443 0.57211128 0.57876771 0.75234682
 0.73050009 0.75337088 0.72776924 0.6740058  0.63918757 0.70080218
 0.57398874 0.73698583 0.59976105 0.6572794  0.62860556 0.58986175
 0.76890254 0.74193548 0.6374808  0.82573818 0.58320533 0.67878478
 0.56989247 0.84297662 0.59856631 0.73306025 0.63253115 0.59515276
 0.56835637 0.68458781 0.59907834 0.71411504 0.72213688 0.58388804
 0.57450077 0.78631166 0.53473289 0.52585765]
Feedback idx at HITL iteration 1: [35 58 10 54 52 44  7  4 30 31 48 50 63 42  5 41 47 21 36 28]
Human score at HITL iteration 1: [0.9867019269430932, 0.00040020526822458246, 0.0020783777576214796, 0.00043659001863693774, 0.002930853529321367, 0.0015023463827720975, 0.0023512561306113

Found local copy...


[0.82881038 0.35279058 0.36286056 0.79501621 0.74125277 0.81532685
 0.79723502 0.51032599 0.75507766 0.70489845 0.79467486 0.66666667
 0.77009729 0.62894692 0.4874552  0.71189623 0.63543267 0.74039939
 0.67673664 0.75285885 0.57381806 0.7380099  0.67383513 0.54906981
 0.75456563 0.74005803 0.65966889 0.57620754 0.58986175 0.65591398
 0.74313023 0.75149343 0.72299027 0.708312   0.59959037 0.67161632
 0.5734767  0.64021164 0.60249189 0.72589179 0.60044376 0.69073221
 0.64481994 0.80559822 0.59822495 0.83836832 0.57040451 0.71786994
 0.59020311 0.79177334 0.60351596 0.64823349 0.5719406  0.59071514
 0.5323434  0.81396143 0.69124424 0.6953405  0.66325311 0.6526711
 0.54480287 0.91175969 0.56835637 0.55111794]
Feedback idx at HITL iteration 2: [29 25  2 39 23 37 24  6 18 11 19  9 51 14  1 57 40 45 27 17]
Human score at HITL iteration 2: [0.002882913847701536, 0.0038778251551519565, 0.042604265409391606, 0.026066280338538337, 0.009149499824961166, 0.00023646306532683795, 0.002113877884627324

Found local copy...


[0.82112989 0.35210787 0.39545998 0.76856119 0.5437788  0.81583888
 0.79996586 0.4889913  0.67178699 0.77504694 0.74415429 0.63270183
 0.6031746  0.67366445 0.70114354 0.71616317 0.55197133 0.7677078
 0.74551971 0.77351084 0.54668032 0.7212835  0.70131422 0.5186892
 0.7463731  0.7151391  0.60829493 0.59259259 0.57091654 0.66461854
 0.75234682 0.7745349  0.73050009 0.65506059 0.58866701 0.64823349
 0.5598225  0.61170848 0.57381806 0.83239461 0.63048302 0.71804062
 0.66001024 0.78716505 0.74773852 0.82607954 0.66581328 0.78187404
 0.60539341 0.82522615 0.60180918 0.72367298 0.57040451 0.69141492
 0.55333675 0.90954088 0.6542072  0.63270183 0.59447005 0.61836491
 0.52773511 0.75354156 0.56596689 0.56630824]
Feedback idx at HITL iteration 3: [34 60 13 53 56 59 22 15 32  3 33 38 43 46  8 49 12 55 16 20]
Human score at HITL iteration 3: [0.009552103978081494, 0.6258949320292332, 0.01671877614658157, 0.0038815126765130705, 0.00042351649440509263, 0.07645445355065277, 0.0053950894575785155, 4.

ValueError: Cannot take a larger sample than population when 'replace=False'